In [2]:
import os, sys
data_dir = os.path.join(os.getcwd(), "data")
from itertools import product

import yaml
import pandas as pd

import pcse
from pcse.fileinput import CABOFileReader
from pcse.models import Wofost72_WLP_FD
from pcse.base import ParameterProvider
from pcse.exceptions import WeatherDataProviderError
from pcse.util import WOFOST72SiteDataProvider
from pcse.db import NASAPowerWeatherDataProvider
from progressbar import *

print("This notebook was built with:")
print("python version: %s " % sys.version)
print("PCSE version: %s" %  pcse.__version__)

This notebook was built with:
python version: 3.9.16 (main, Mar  8 2023, 10:39:24) [MSC v.1916 64 bit (AMD64)] 
PCSE version: 5.5.5


In [16]:
crop_dir = os.path.join(data_dir, 'crop')
crop_files = [CABOFileReader(os.path.join(crop_dir,f)) for f in os.listdir(crop_dir) if f.endswith("CAB")]

In [17]:
soil_dir = os.path.join(data_dir, 'soil')
soil_files = [CABOFileReader(os.path.join(soil_dir,f)) for f in os.listdir(soil_dir) if f.startswith("ec")]

In [18]:
sited = WOFOST72SiteDataProvider(WAV=10, CO2=360., SMLIM=0.35)

Unknown parameter values provided to WOFOSTSiteDataProvider: {'CO2': 360.0}


In [19]:
weatherdata = NASAPowerWeatherDataProvider(longitude=5, latitude=52)
print(weatherdata)

Weather data provided by: NASAPowerWeatherDataProvider
--------Description---------
NASA/POWER CERES/MERRA2 Native Resolution Daily Data
----Site characteristics----
Elevation:    3.5
Latitude:  52.000
Longitude:  5.000
Data available for 1984-01-01 - 2023-05-04
Number of missing days: 0



In [20]:
df = pd.DataFrame(weatherdata.export()).set_index("DAY")
df.tail()

,LAT,LON,ELEV,IRRAD,TMIN,TMAX,VAP,RAIN,E0,ES0,ET0,WIND,TEMP
DAY,,,,,,,,,,,,,
2023-04-30,52.0,5.0,3.51,22230000.0,4.00,17.51,10.339473,0.003,0.334506,0.283498,0.293295,2.08,11.32
2023-05-01,52.0,5.0,3.51,10880000.0,7.98,17.18,10.911722,0.059,0.224512,0.198396,0.216550,1.91,12.01
2023-05-02,52.0,5.0,3.51,17720000.0,3.65,13.30,8.387938,0.006,0.281637,0.243394,0.244119,2.97,9.16
2023-05-03,52.0,5.0,3.51,21880000.0,2.69,14.98,8.013186,0.000,0.335086,0.287388,0.298381,2.70,9.30
2023-05-04,52.0,5.0,3.51,20960000.0,5.12,20.58,11.227160,0.000,0.408632,0.357999,0.346123,3.18,13.11


In [21]:
agro_maize = """
- {year}-03-01:
    CropCalendar:
        crop_name: '{crop}'
        variety_name: 'maize'
        crop_start_date: {year}-04-15
        crop_start_type: sowing
        crop_end_date:
        crop_end_type: maturity
        max_duration: 300
    TimedEvents: null
    StateEvents: null
- {year}-12-01: null
"""

agro_potato = """
- {year}-03-01:
    CropCalendar:
        crop_name: '{crop}'
        variety_name: 'potato'
        crop_start_date: {year}-05-01
        crop_start_type: sowing
        crop_end_date:
        crop_end_type: maturity
        max_duration: 300
    TimedEvents: null
    StateEvents: null
- {year}-12-01: null
"""

agro_sugarbeet = """
- {year}-03-01:
    CropCalendar:
        crop_name: '{crop}'
        variety_name: 'sugar_beet'
        crop_start_date: {year}-05-01
        crop_start_type: sowing
        crop_end_date: {year}-10-15
        crop_end_type: harvest
        max_duration: 300
    TimedEvents: null
    StateEvents: null
- {year}-12-01: null
"""
agro_templates = [agro_maize, agro_potato, agro_sugarbeet]

In [23]:
# Placeholder for storing summary results
summary_results = []

# Years to simulate
years = range(2004, 2008)

# Loop over crops, soils and years
crops_calendars = zip(crop_files, agro_templates)
all_runs = product(crops_calendars, soil_files, years)
nruns = len(years) * len(crop_files) * len(soil_files)

progressbar.ProgressBar(maxval = 0)
for i, inputs in enumerate(all_runs):
    (cropd, agro), soild, year = inputs
    crop_type = cropd['CRPNAM']
    soil_type = soild['SOLNAM']
    # String to identify this run
    run_id = "{crop}_{soil}_{year}".format(crop=crop_type, soil=soil_type, year=year)

    # Set the agromanagement with correct year and crop
    agromanagement = yaml.load(agro.format(year=year, crop=crop_type))

    # Encapsulate parameters
    parameters = ParameterProvider(sitedata=sited, soildata=soild, cropdata=cropd)

    # Start WOFOST, run the simulation 
    try:
        wofost = Wofost72_WLP_FD(parameters, weatherdata, agromanagement)
        wofost.run_till_terminate()
    except WeatherDataProviderError as e:
        msg = "Runid '%s' failed because of missing weather data." % run_id
        print(msg)
        continue
    progressbar.ProgressBar(i+1, nruns, prefix = 'Progress:', suffix = 'Complete', length = 50)

    # convert daily output to Pandas DataFrame and store it
    df = pd.DataFrame(wofost.get_output()).set_index("day")
    fname = os.path.join(data_dir, "output", run_id + ".xlsx")
    df.to_excel(fname)
    # Collect summary results
    r = wofost.get_summary_output()[0]
    r['run_id'] = run_id
    summary_results.append(r)

TypeError: load() missing 1 required positional argument: 'Loader'